In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
X= dataset.drop('expenses', axis=1).values  # Features
y = dataset['expenses'].values               # Labels


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
from sklearn.preprocessing import MinMaxScaler

feature_scaler = MinMaxScaler()
label_scaler = MinMaxScaler()

X_train_scaled = feature_scaler.fit_transform(X_train)
X_test_scaled = feature_scaler.transform(X_test)

y_train_scaled = label_scaler.fit_transform(y_train.reshape(-1,1))
y_test_scaled = label_scaler.transform(y_test.reshape(-1,1))


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model=tf.keras.Sequential([
    Dense(128,activation='relu',input_shape=(X_train_scaled.shape[1],)),
    Dense(64,activation='relu'),
    Dense(32,activation='relu'),
    Dense(64,activation='relu'),
    Dense(1)

])
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])
history=model.fit(
    X_train_scaled, y_train_scaled,
    validation_split=0.2,
    epochs=10,
    batch_size=32,
    verbose=2
)

In [ ]:
# Evaluate the model on test data
loss, mae = model.evaluate(X_test_scaled, y_test_scaled, verbose=2)
print("Scaled testing set Mean Abs Error:", mae)

# Convert predictions back to original scale
test_predictions_scaled = model.predict(X_test_scaled)
test_predictions = label_scaler.inverse_transform(test_predictions_scaled)
y_test_orig = label_scaler.inverse_transform(y_test_scaled)

# Compute MAE in original units
from sklearn.metrics import mean_absolute_error
mae_orig = mean_absolute_error(y_test_orig, test_predictions)
print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae_orig))

if mae_orig < 3500:
    print("You passed the challenge. Great job!")
else:
    print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions
import matplotlib.pyplot as plt

a = plt.axes(aspect='equal')
plt.scatter(y_test_orig, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)
